In [1]:
from symbolica import Expression, S,E
from symbolica.community.spenso import TensorName as N,LibraryTensor,TensorNetwork,Representation,TensorStructure,TensorIndices,Tensor,Slot,TensorLibrary

# import symbolica.community.spenso as tensors
import random

# Tensor structures and representations:

The starting point for defining a tensor is to define its structure. This is done by defining the indices that the tensor has. Each index is in fact a slot, built from an index and a representation

Representations are defined by their name and dimension. They can be self-dual or not.


In [2]:
mink = Representation.mink(4)
bis = Representation.bis(4)
custom = Representation("custom",4,is_self_dual=False)
print(custom)

custom(4)


Slots are created from a representation and an index

In [3]:
mu = mink("mu")
print(mu)
mu

mink(4,mu)


Slot { rep: Representation { rep: InlineMetric(0), dim: Concrete(4) }, aind: Symbol(SerializableSymbol { symbol: 79 }) }

They can be converted to symbolica expressions

In [4]:
mue = mu.to_expression()
mue


spenso::mink(4,spynso3::mu)

In [5]:
nu = Slot("mink",4,"nu")

In [6]:
# The index can be a string (that could be parsed into a symbolica symbol)
i = bis("i")
# The index can also be an integer
j = bis(2)

k = S("k")
# The index can also directly a symbolica expression
k = bis(k)
print(k)
k

bis(4,k)


Slot { rep: Representation { rep: SelfDual(1), dim: Concrete(4) }, aind: Symbol(SerializableSymbol { symbol: 82 }) }

In [7]:
gamma = N.gamma
p = N("P")
w = N("w")
g = N.g
mq = S("mq")

Tensor structures are essentially a list of slots, with a name (symbolica symbol)
It can be turned into a symbolica expression


In [8]:

other_g = gamma(i,k,mu)
g_muik = TensorIndices(i,k,mu,name=gamma)
print(g_muik)
print(other_g)
# g_muik

gamma(bis(4,i),bis(4,k),mink(4,mu))
gamma(bis(4,i),bis(4,k),mink(4,mu))


In [9]:
g_muik[2]

[0, 0, 2]

In [10]:
print(gamma(k,j,mu).to_expression())

gamma(bis(4,k),bis(4,2),mink(4,mu))


In [11]:

g_muik[45:63:3]


[[2, 3, 1], [3, 0, 0], [3, 0, 3], [3, 1, 2], [3, 2, 1], [3, 3, 0]]

In [12]:
g_muik[[2,2,2]]

42

In [13]:
x = g_muik*(p(2,nu)*gamma(k,j,nu)+mq*g(k,j))*w(1,i)*w(3,mu)

print(x.to_canonical_string())

(python::mq*spenso::g(spenso::bis(4,2),spenso::bis(4,python::k))+spenso::gamma(spenso::bis(4,python::k),spenso::bis(4,2),spenso::mink(4,spynso3::nu))*spenso_python::P(2,spenso::mink(4,spynso3::nu)))*spenso::gamma(spenso::bis(4,spynso3::i),spenso::bis(4,python::k),spenso::mink(4,spynso3::mu))*spenso_python::w(1,spenso::bis(4,spynso3::i))*spenso_python::w(3,spenso::mink(4,spynso3::mu))


In [14]:
tn = TensorNetwork(x)
# prints the rich graph associated to the network
print(tn)

digraph {
  node	 [shape=circle,height=0.1,label=""];
  overlap = "scale";
  layout = "neato";

  0	 [label = "∏"];
  1	 [label = "∑"];
  2	 [label = "∏"];
  3	 [label = "S:0:mq"];
  4	 [label= "L:g()"];
  5	 [label = "∏"];
  6	 [label= "L:gamma()"];
  7	 [label = "T:P(2)"];
  8	 [label= "L:gamma()"];
  9	 [label = "T:w(1)"];
  10	 [label = "T:w(3)"];
  ext0	 [style=invis];
  0:0:s	-> ext0	 [id=0 color="red"];
  8:33:s	-> 10:37:s	 [id=1 dir=none  color="red:blue;0.5" label="mink4|mu"];
  8:31:s	-> 9:35:s	 [id=2 dir=none  color="red:blue;0.5" label="bis4|i"];
  1:11:s	-> 8:32:s	 [id=3 dir=none  color="red:blue;0.5" label="bis4|k"];
  5:21:s	-> 1:6:s	 [id=4  color="red:blue;0.5"];
  6:27:s	-> 7:29:s	 [id=5 dir=none  color="red:blue;0.5" label="mink4|nu"];
  6:25:s	-> 1:9:s	 [id=6 dir=none  color="red:blue;0.5" label="bis4|2"];
  4:19:s	-> 1:10:s	 [id=7 dir=none  color="red:blue;0.5" label="bis4|2"];
  ext8	 [style=invis];
  1:8:s	-> ext8	 [id=8 dir=none color="red" label="bis4|2"];
  6:2

As you can see when parsed, the network isn't contracted yet, so it's just a graph of the expression
To contract it, you can call the contract method. You can specify the contraction procedure with optional arguments. 


In [15]:
from symbolica.community.spenso import ExecutionMode
tn.execute(n_steps=2,mode=ExecutionMode.Scalar)


In [16]:
t = TensorNetwork.one()*TensorNetwork.zero() + TensorNetwork.one()*TensorNetwork.zero()
print(t)
t.execute()
print(t)

digraph {
  node	 [shape=circle,height=0.1,label=""];
  overlap = "scale";
  layout = "neato";

  0	 [label = "∑"];
  1	 [label = "∏"];
  2	 [label = "∑"];
  3	 [label = "∏"];
  4	 [label = "∏"];
  5	 [label = "∑"];
  6	 [label = "∏"];
  ext0	 [style=invis];
  0:0:s	-> ext0	 [id=0 color="red"];
  6:12:s	-> 4:9:s	 [id=1  color="red:blue;0.5"];
  3:7:s	-> 1:4:s	 [id=2  color="red:blue;0.5"];
  2:6:s	-> 1:5:s	 [id=3  color="red:blue;0.5"];
  1:3:s	-> 0:2:s	 [id=4  color="red:blue;0.5"];
  5:11:s	-> 4:10:s	 [id=5  color="red:blue;0.5"];
  4:8:s	-> 0:1:s	 [id=6  color="red:blue;0.5"];
}

digraph {
  node	 [shape=circle,height=0.1,label=""];
  overlap = "scale";
  layout = "neato";

  0	 [label = "∑"];
  3	 [label = "∏"];
  1	 [label = "∑"];
  2	 [label = "∑"];
  4	 [label = "∏"];
  ext0	 [style=invis];
  0:0:s	-> ext0	 [id=0 color="red"];
  4:8:s	-> 0:1:s	 [id=1  color="red:blue;0.5"];
  1:4:s	-> 4:7:s	 [id=2  color="red:blue;0.5"];
  2:6:s	-> 3:5:s	 [id=3  color="red:blue;0.5"];
  3:3:s	->

The graph has now reduced in size

In [17]:
print(tn)


digraph {
  node	 [shape=circle,height=0.1,label=""];
  overlap = "scale";
  layout = "neato";

  0	 [label = "∏"];
  1	 [label = "∑"];
  2	 [label = "∏"];
  3	 [label = "S:0:mq"];
  4	 [label= "L:g()"];
  5	 [label = "T:w(3)"];
  6	 [label = "T:w(1)"];
  7	 [label= "L:gamma()"];
  8	 [label = "T:P(2)"];
  9	 [label= "L:gamma()"];
  10	 [label = "∏"];
  ext0	 [style=invis];
  0:0:s	-> ext0	 [id=0 color="red"];
  1:5:s	-> 0:4:s	 [id=1  color="red:blue;0.5"];
  9:33:s	-> 1:9:s	 [id=2 dir=none  color="red:blue;0.5" label="bis4|2"];
  ext3	 [style=invis];
  1:8:s	-> ext3	 [id=3 dir=none color="red" label="bis4|2"];
  9:32:s	-> 1:12:s	 [id=4 dir=none  color="red:blue;0.5" label="bis4|k"];
  1:11:s	-> 7:26:s	 [id=5 dir=none  color="red:blue;0.5" label="bis4|k"];
  9:34:s	-> 10:35:s	 [id=6  color="red:blue;0.5"];
  2:14:s	-> 1:7:s	 [id=7  color="red:blue;0.5"];
  3:17:s	-> 2:16:s	 [id=8  color="red:blue;0.5"];
  4:18:s	-> 2:15:s	 [id=9  color="red:blue;0.5"];
  4:19:s	-> 1:10:s	 [id=10 dir=no

We can now extract the resulting tensor, by fully executing the network


In [18]:
tn.execute()
t = tn.result_tensor()


The tensor is a tensor object


In [19]:
t


Spensor(
() [0]() [0][0]: ((-w(3,cind(1))-1𝑖*w(3,cind(2)))*(P(2,cind(0))+P(2,cind(3)))+(P(2,cind(1))+1𝑖*P(2,cind(2)))*(w(3,cind(0))+w(3,cind(3))))*w(1,cind(1))+((-w(3,cind(1))+1𝑖*w(3,cind(2)))*(P(2,cind(1))+1𝑖*P(2,cind(2)))+(P(2,cind(0))+P(2,cind(3)))*(w(3,cind(0))-w(3,cind(3))))*w(1,cind(0))+mq*(w(3,cind(0))+w(3,cind(3)))*w(1,cind(2))+mq*(w(3,cind(1))+1𝑖*w(3,cind(2)))*w(1,cind(3))
[1]: ((-w(3,cind(1))-1𝑖*w(3,cind(2)))*(P(2,cind(1))-1𝑖*P(2,cind(2)))+(P(2,cind(0))-P(2,cind(3)))*(w(3,cind(0))+w(3,cind(3))))*w(1,cind(1))+((-w(3,cind(1))+1𝑖*w(3,cind(2)))*(P(2,cind(0))-P(2,cind(3)))+(P(2,cind(1))-1𝑖*P(2,cind(2)))*(w(3,cind(0))-w(3,cind(3))))*w(1,cind(0))+mq*(w(3,cind(0))-w(3,cind(3)))*w(1,cind(3))+mq*(w(3,cind(1))-1𝑖*w(3,cind(2)))*w(1,cind(2))
[2]: ((-P(2,cind(1))-1𝑖*P(2,cind(2)))*(w(3,cind(0))-w(3,cind(3)))+(P(2,cind(0))-P(2,cind(3)))*(w(3,cind(1))+1𝑖*w(3,cind(2))))*w(1,cind(3))+((-P(2,cind(1))-1𝑖*P(2,cind(2)))*(w(3,cind(1))-1𝑖*w(3,cind(2)))+(P(2,cind(0))-P(2,cind(3)))*(w(3,cind(0))+w(3,ci

 It has a structure


In [20]:
print(t.structure())


(bis(4,2))


# Evaluation of the tensor

 You may have noticed that the resulting tensor is a set of expressions with certain functions that label the 'concrete' values of the tensor. What if we want to evaluate the tensor for a given set of parameters?


In [21]:

params = [Expression.I]
params += TensorNetwork(w(1,i)).result_tensor() # tensors implement the sequence protocol, so can be treated just like lists
params += TensorNetwork(w(3,mu)).result_tensor()
params += TensorNetwork(p(2,nu)).result_tensor()
constants = {mq: E("173")}

# Much like the expressions, tensors have the same evaluation api, just that they return a tensor instead of an expression
e=t.evaluator(constants=constants, params=params, funs={})
# The evaluator can be compiled to a shared library
c = e.compile(function_name="f", filename="test_expression.cpp",
              library_name="test_expression.so", inline_asm="none")


e_params = [random.random()+1j*random.random() for i in range(len(params))]
eval_res = e.evaluate_complex([e_params])[0]

print(eval_res)
print(eval_res.structure())


[0]: (26.691751661999817+282.1430839021581i)
[1]: (133.61773361329213+131.40687432939498i)
[2]: (191.02246134986044+-136.9589355210573i)
[3]: (-130.2734351522607+108.68819064880141i)

(bis(4,2))



# Tensor building:
 Tensors have two storage modes. Hashmap backed sparse tensors and dense tensors. Sparse tensors are built from a structure and then assigned values:


In [22]:


t = LibraryTensor.sparse([custom,custom],type(mq))
# Note that the structure is a list of representations, not slots
# In this case the tensor structure is a `TensorStructure` object
print(t.structure())


[custom(4),custom(4)]


It does not have indices, just a shape. This makes sense if you just want to register this tensor (see later), or if you just want to use it for storage and iteration.
The tensor can store expressions,floats or complex numbers (homogeneously)
values are accessed by flattened index: (and slices are supported)


In [23]:
t[6]=E("f(x)*(1+y)")
t

Spensor(
() [0 1]() [0 1][1, 2]: (y+1)*f(x)
)

Or by multi-index:


In [24]:
t[[3,2]]=E("sin(alpha)")
t


Spensor(
() [0 1]() [0 1][1, 2]: (y+1)*f(x)
[3, 2]: sin(alpha)
)

Currently printing makes more sense when the data is dense

In [25]:
t.to_dense()
t


Spensor(
() [0 1]() [0 1][0, 0]: 0
[0, 1]: 0
[0, 2]: 0
[0, 3]: 0
[1, 0]: 0
[1, 1]: 0
[1, 2]: (y+1)*f(x)
[1, 3]: 0
[2, 0]: 0
[2, 1]: 0
[2, 2]: 0
[2, 3]: 0
[3, 0]: 0
[3, 1]: 0
[3, 2]: sin(alpha)
[3, 3]: 0
)

# Registering to a library

In [26]:

d = Representation("newrep",3)
tname = S("test")
#  Dense tensors are built from a list of values in row-major order.
t = LibraryTensor.dense(TensorStructure(d,d,name=tname),[0,0,123,
                        11,3,234,
                        234,23,44,])
# If the structure is just a list of integers, it is assumed to be a list of dimensions, and the representation is assumed to be the default representation: euclidean.
t[[1,2]]=3/34
print(t)
print(t.structure())

lib = TensorLibrary.hep_lib()

lib.register(t)

new_t = t.structure()


x=(new_t(1,2)*new_t(2,3)*new_t(3,1))
n =  TensorNetwork(x,library=lib)
n.execute(library=lib)
t = n.result_tensor(library=lib)
print(t)


() [0 1]() [0 1][0, 0]: 0
[0, 1]: 0
[0, 2]: 123
[1, 0]: 11
[1, 1]: 3
[1, 2]: 0.08823529411764706
[2, 0]: 234
[2, 1]: 23
[2, 2]: 44

test[newrep(3),newrep(3)]
[]: 3978078.147058823



# Algebraic simplification

In [27]:
ag = S("spenso::gamma")
minkd = Representation("mink","D")
fc = S("spenso::f")
ps = S("p")
coad = Representation("coad",8)

def to_expression(t: Expression | Slot) -> Expression:
    if isinstance(t,Expression):
        return t
    elif isinstance(t,Slot):
        return t.to_expression()
    else:
        raise TypeError(f"Expected Expression or Slot, got {type(t)}")

gam = lib["spenso::gamma"]
def p(i):
    m  = to_expression(minkd(i))
    return ps(m,)

def f(i,j,k):
    return fc(to_expression(coad(i)),to_expression(coad(j)),to_expression(coad(k)))


In [28]:
from symbolica.community.idenso import simplify_color, simplify_metrics, simplify_gamma, to_dots

print(simplify_metrics(bis.g(4,2)*gam(2,3,1)))

gamma(bis(4,4),bis(4,3),mink(4,1))


In [29]:
simplify_metrics(bis.g(1,1).to_expression())

4

In [30]:
simplify_metrics(Representation.euc("d").g(1,1).to_expression())

spenso_python::d

In [31]:
a = simplify_gamma(gam(1,2,1)*gam(2,3,"mu")*gam(3,4,1)*gam(4,1,2)*p("mu")*p(2))

In [32]:
print(to_dots(a))

-8*g(mink(4,2),mink(4,mu))*p(mink(D,2))*p(mink(D,mu))


In [ ]:
simplify_color(f(1,2,3)*f(3,2,1))

-16*spenso::TR*spenso::Nc